In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold



from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.preprocessing import QuantileTransformer

In [3]:

data_dir = '../data/01_raw'
os.listdir(data_dir)

['train_targets_scored.csv',
 'sample_submission.csv',
 '.gitkeep',
 'train_drug.csv',
 'train_features.csv',
 'test_features.csv',
 'train_targets_nonscored.csv']

In [4]:
# Parameters
no_ctl = True
ncompo_genes = 600
ncompo_cells = 50

In [5]:
train_features = pd.read_csv(data_dir+'/train_features.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir+'/train_targets_nonscored.csv')

test_features = pd.read_csv(data_dir+'/test_features.csv')
sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')
drug = pd.read_csv(data_dir+'/train_drug.csv')

In [6]:
train_targets_scored[train_targets_scored["sig_id"] == "id_fffcb9e7c"]

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
targets_nscored = train_targets_nonscored.columns[1:]
nscored = train_targets_nonscored.merge(drug, on='sig_id', how='left') 

In [8]:
if no_ctl:
    # cp_type == ctl_vehicle
    print("not_ctl")
    train_features = train_features[train_features["cp_type"] != "ctl_vehicle"]
    test_features = test_features[test_features["cp_type"] != "ctl_vehicle"]
    train_targets_nonscored = train_targets_nonscored.iloc[train_features.index]
    train_features.reset_index(drop = True, inplace = True)
    test_features.reset_index(drop = True, inplace = True)
    train_targets_nonscored.reset_index(drop = True, inplace = True)

not_ctl


In [9]:
# #Indiquer si valeur dans le range max, min

# import seaborn as sns
# data = pd.concat([train_features,test_features],axis=0)

# sns.distplot(data[data["cp_type"] == "ctl_vehicle"]["c-4"],label="normal")

# sns.distplot(data[data["cp_type"] == "trt_cp"]["c-4"],label="treated")
# plt.legend()
# plt.show()

In [10]:
# sum_targets = train_targets_scored[[c for c in train_targets_scored.columns if (c != "sig_id")]].sum().reset_index()

In [11]:
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_inhibitor",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_activator",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_agonist",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_antagonist",""))

In [12]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [13]:
# from scipy import stats

# train_features[GENES].apply(lambda x : stats.moment(x,moment=5),axis=1)

In [14]:
seed=42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed)

In [15]:
#RankGauss
for col in (GENES + CELLS):
    transformer = QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

#True gauss rank
# import cupy as cp
# from cupyx.scipy.special import erfinv
# epsilon = 1e-6

# data = pd.concat([pd.DataFrame(train_features[GENES + CELLS]), pd.DataFrame(test_features[GENES + CELLS])])

# for k in (GENES + CELLS):
#     r_gpu = cp.array(data.loc[:,k])
#     r_gpu = r_gpu.argsort().argsort()
#     r_gpu = (r_gpu/r_gpu.max()-0.5)*2 
#     r_gpu = cp.clip(r_gpu,-1+epsilon,1-epsilon)
#     r_gpu = erfinv(r_gpu) 
#     data.loc[:,k] = cp.asnumpy( r_gpu * np.sqrt(2) )

# train_features[GENES + CELLS] = data[:train_features.shape[0]]; test_features[GENES + CELLS] = data[-test_features.shape[0]:]

In [16]:
# GENES
n_comp = ncompo_genes 

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[train_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [17]:
#CELLS
n_comp = ncompo_cells

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [18]:
from scipy import stats

def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1) ## <==
        # df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1) ## <==
        # df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        #df['g_skew'] = df[features_g].skew(axis = 1)
        # df['g_q25'] = df[features_g].quantile(q=.25,axis = 1)
        # df['g_q50'] = df[features_g].quantile(q=.5,axis = 1)
        # df['g_q75'] = df[features_g].quantile(q=.75,axis = 1)
        #df['g_var'] = df[features_g].apply(axis=1,func=stats.variation)
        # df['g_mad'] = df[features_g].mad(axis = 1)


        df['c_sum'] = df[features_c].sum(axis = 1) ## <==
        # df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1) ## <==
        # df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        #df['c_skew'] = df[features_c].skew(axis = 1)
        # df['c_q25'] = df[features_c].quantile(q=.25,axis = 1)
        # df['c_q50'] = df[features_c].quantile(q=.5,axis = 1)
        # df['c_q75'] = df[features_c].quantile(q=.75,axis = 1)
        # df['c_var'] = df[features_c].apply(axis=1,func=stats.variation)
        # df['c_mad'] = df[features_c].mad(axis = 1)



        df['gc_sum'] = df[features_g + features_c].sum(axis = 1) ## <==
        # df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        # df['gc_std'] = df[features_g + features_c].std(axis = 1)
        # df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        # df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        # df['gc_q25'] = df[features_g + features_c].quantile(q=.25,axis = 1)
        # df['gc_q50'] = df[features_g + features_c].quantile(q=.5,axis = 1)
        # df['gc_q75'] = df[features_g + features_c].quantile(q=.75,axis = 1)
        # df['gc_var'] = df[features_g + features_c].apply(axis=1,func=stats.variation)
        # df['gc_mad'] = df[features_g + features_c].mad(axis = 1)


        
    return train, test

train_features,test_features=fe_stats(train_features,test_features)

In [19]:
# train_features_gc = train_features[["sig_id"]+GENES+CELLS].copy()
# test_features_gc = test_features[["sig_id"]+GENES+CELLS].copy()

In [20]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(0.8)  #<-- Update
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(21948, 1047)

In [21]:
# from sklearn.cluster import KMeans
# def fe_cluster2(train, test, n_clusters = 3, SEED = 42):
    

#     def create_cluster(train, test, n_clusters = n_clusters):
#         train_ = train.copy()
#         test_ = test.copy()
#         data = pd.concat([train_, test_], axis = 0)
#         kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data[[c for c in data.columns if c not in ["sig_id","cp_type","cp_dose","cp_time"]]])
#         train['cluster'] = kmeans.labels_[:train.shape[0]]
#         test['cluster'] = kmeans.labels_[train.shape[0]:]
#         train = pd.get_dummies(train, columns = ['cluster'])
#         test = pd.get_dummies(test, columns = ['cluster'])
#         return train, test
    
#     train, test = create_cluster(train, test, n_clusters = n_clusters)
#     return train, test



# train_features,test_features=fe_cluster2(train_features,test_features)

In [22]:
# data = pd.concat([train1, test1], axis = 0)

# distortion = []
# for k in range(1,10):
#     kmeans = KMeans(n_clusters = k, random_state = 42).fit(data)
#     distortion += [kmeans.inertia_]


In [23]:
# plt.plot(range(1,10),distortion)

In [24]:
# train_features = train_features.merge(train_features_gc.loc[:,[col for col in train_features_gc.columns if col not in GENES+CELLS]],on="sig_id")
# test_features = test_features.merge(test_features_gc.loc[:,[col for col in test_features_gc.columns if col not in GENES+CELLS]],on="sig_id")

In [25]:
train = train_features.merge(train_targets_nonscored, on='sig_id')
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target_nscored = train[train_targets_nonscored.columns]

In [26]:
train = train.drop('cp_type', axis=1)
test = test_features.drop('cp_type', axis=1)

In [27]:
target_nscored_cols = target_nscored.drop('sig_id', axis=1).columns.values.tolist()

In [28]:
# folds = train.copy()

# mskf = MultilabelStratifiedKFold(n_splits=7)

# for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target_scored)):
#     folds.loc[v_idx, 'kfold'] = int(f)

# folds['kfold'] = folds['kfold'].astype(int)
# folds

In [29]:
class TrainDataset:
    def __init__(self, features, targets_nscored):
        self.features = features
        self.targets_nscored = targets_nscored
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y_nscored' : torch.tensor(self.targets_nscored[idx, :], dtype=torch.float)           
        }
        return dct

class ValidDataset:
    def __init__(self, features, targets_nscored):
        self.features = features
        self.targets_nscored = targets_nscored
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y_nscored' : torch.tensor(self.targets_nscored[idx, :], dtype=torch.float),          
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [30]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y_nscored'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y_nscored'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        

    final_loss /= len(dataloader)


    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds_nscored = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs_nscored = model(inputs)
        
        preds_nscored.append(outputs_nscored.sigmoid().detach().cpu().numpy())

    preds_nscored = np.concatenate(preds_nscored)
    
    return preds_nscored

In [31]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [32]:
class Model(nn.Module):
    def __init__(self, num_features, 
                 num_targets_nscored, 
                 hidden_sizes,
                 dropout_rates):

        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_sizes[0]))
        self.activation1 = torch.nn.PReLU(num_parameters = hidden_sizes[0], init = 1.0)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_sizes[0])
        self.dropout2 = nn.Dropout(dropout_rates[0])
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_sizes[0], hidden_sizes[1]))
        self.activation2 = torch.nn.PReLU(num_parameters = hidden_sizes[1], init = 1.0)


        # self.batch_norm2b = nn.BatchNorm1d(hidden_sizes[1])
        # self.dropout2b = nn.Dropout(dropout_rates[1])
        # self.dense2b = nn.utils.weight_norm(nn.Linear(hidden_sizes[1], hidden_sizes[2]))
        # self.activation2b = torch.nn.PReLU(num_parameters = hidden_sizes[2], init = 1.0)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_sizes[1])
        self.dropout3 = nn.Dropout(dropout_rates[1])
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_sizes[1], num_targets_nscored))

    def init_bias(self,pos_scored_rate,pos_nscored_rate):
        self.dense3.bias.data = nn.Parameter(torch.tensor(pos_scored_rate, dtype=torch.float))
        self.dense4.bias.data = nn.Parameter(torch.tensor(pos_nscored_rate, dtype=torch.float))
    
    def recalibrate_layer(self, layer):
        if(torch.isnan(layer.weight_v).sum() > 0):
            print ('recalibrate layer.weight_v')
            layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
            layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

        if(torch.isnan(layer.weight).sum() > 0):
            print ('recalibrate layer.weight')
            layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
            layer.weight += 1e-7

    def forward(self, x):
        x = self.batch_norm1(x)
        self.recalibrate_layer(self.dense1)
        x = self.activation1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        self.recalibrate_layer(self.dense2)
        x = self.activation2(self.dense2(x))

        # x = self.batch_norm2b(x)
        # x = self.dropout2b(x)
        # self.recalibrate_layer(self.dense2b)
        # x = self.activation2b(self.dense2b(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        self.recalibrate_layer(self.dense3)
        x = self.dense3(x)

        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [33]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [34]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 30
#EPOCHS = 300 #200
PATIENCE=40

BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 7           
EARLY_STOPPING_STEPS = PATIENCE+5
EARLY_STOP = False

#hidden_size=1500
hidden_sizes = [1300,1100]
dropout_rates = [0.2619422201258426,0.2619422201258426]
#dropout_rate = 0.2619422201258426
#dropout_rate = 0.28


In [35]:
# LOCATE DRUGS
vc = nscored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = nscored.groupby('drug_id')[targets_nscored].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_nscored])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = nscored.loc[nscored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_nscored])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
folds = train.merge(drug,on="sig_id")
folds['fold'] = folds.drug_id.map(dct1)
folds.loc[folds.fold.isna(),'fold'] =\
    folds.loc[folds.fold.isna(),'sig_id'].map(dct2)
folds.fold = folds.fold.astype('int8')

In [36]:
feature_cols = [c for c in process_data(train).columns if c not in (target_nscored_cols)]
feature_cols = [c for c in feature_cols if c not in ['fold','sig_id']]
len(feature_cols)

1048

In [37]:
num_features=len(feature_cols)
num_targets_nscored=len(target_nscored_cols)

In [38]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target_nscored.shape)
print(sample_submission.shape)

(21948, 1448)
(21948, 1450)
(3624, 1046)
(21948, 403)
(3982, 207)


In [39]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)

    
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    x_train, y_nscored_train  = train_df[feature_cols].values, train_df[target_nscored_cols].values
    x_valid, y_nscored_valid =  valid_df[feature_cols].values, valid_df[target_nscored_cols].values
    
    train_dataset = TrainDataset(x_train, y_nscored_train)
    valid_dataset = ValidDataset(x_valid, y_nscored_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets_nscored=num_targets_nscored,
        hidden_sizes=hidden_sizes,
        dropout_rates=dropout_rates,
    )

    # model.init_bias(pos_scored_rate,pos_nscored_rate)

    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,pct_start=0.1, div_factor=1e4, final_div_factor=1e5,
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))


    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode = "min", patience = PATIENCE, min_lr = 1e-6, factor = 0.9)
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target_nscored.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")
        #scheduler.step(valid_loss)
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_m1_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets_nscored=num_targets_nscored,
        hidden_sizes=hidden_sizes,
        dropout_rates=dropout_rates
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_m1_.pth"))
    model.to(DEVICE)
    
    predictions_nscored = np.zeros((len(test_), target_nscored.iloc[:, 1:].shape[1]))
    predictions_nscored = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions_nscored

In [40]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_nscored_cols)))
    predictions_nscored = np.zeros((len(test), len(target_nscored_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_nscored_ = run_training(fold, seed)
        
        predictions_nscored += pred_nscored_ / NFOLDS
        oof += oof_
        
    return oof, predictions_nscored

In [41]:
# Averaging on multiple SEEDS

#SEED = [0,1,2,3,4,5,6] #<-- Update
SEED = [0]
oof = np.zeros((len(train), len(target_nscored_cols)))
predictions_nscored = np.zeros((len(test), len(target_nscored_cols)))


for seed in SEED:
    
    oof_, predictions_nscored_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions_nscored += predictions_nscored_ / len(SEED)

train[target_nscored_cols] = oof
test[target_nscored_cols] = predictions_nscored

FOLD: 0, EPOCH: 0, train_loss: 0.5416978941533436, valid_loss: 0.027790090590715407
FOLD: 0, EPOCH: 1, train_loss: 0.011876989916271093, valid_loss: 0.005193247105926275
FOLD: 0, EPOCH: 2, train_loss: 0.008628546276136118, valid_loss: 0.005190843120217324
FOLD: 0, EPOCH: 3, train_loss: 0.008594813769315781, valid_loss: 0.007981017380952835
FOLD: 0, EPOCH: 4, train_loss: 0.008569205667646159, valid_loss: 0.005116049442440271
FOLD: 0, EPOCH: 5, train_loss: 0.008293163623711486, valid_loss: 0.004582149498164653
FOLD: 0, EPOCH: 6, train_loss: 0.00826709768513129, valid_loss: 0.00493538349866867
FOLD: 0, EPOCH: 7, train_loss: 0.008251061194202526, valid_loss: 0.004808509293943644
FOLD: 0, EPOCH: 8, train_loss: 0.008246377639497827, valid_loss: 0.004856785433366895
FOLD: 0, EPOCH: 9, train_loss: 0.008255078362262979, valid_loss: 0.004853619318455457
FOLD: 0, EPOCH: 10, train_loss: 0.008262035620658576, valid_loss: 0.004837706182152033
FOLD: 0, EPOCH: 11, train_loss: 0.008271684432972451, val

In [42]:
valid_results = train_targets_nonscored.drop(columns=target_nscored_cols).merge(train[['sig_id']+target_nscored_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_nonscored[target_nscored_cols].values
y_pred = valid_results[target_nscored_cols].values

score = 0
for i in range(len(target_nscored_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i],labels=[0,1])
    score += score_ / target_nscored.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.0047653734598364535


In [43]:
#CV log_loss:  0.004804078075531289
#CV log_loss:  0.004795348601078446
#CV log_loss:  0.0047653734598364535

In [45]:
sub_nscored = sample_submission[["sig_id"]].merge(test[train_targets_nonscored.columns], on='sig_id', how='left').fillna(0)
sub_nscored.to_csv('sub_nscored.csv', index=False)